<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://www.islas.org.mx/">     
           <img 
               src="assets/geci_20_anos.png" 
               style="width:200px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Erradicación de gatos en Socorro</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2> Gatos remanentes</h2></div>

In [ ]:
from pymc import MCMC
import bokeh.io as bi
import bokeh.models as bm
import bokeh.plotting as bp
import gatos as dmc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import gatos.PopulationEstimator as pe

In [ ]:
directorio_dp = ('../inst/extdata/erradicaciones-mamiferos/')
recurso = 'captura_gatos.csv'
DatosSocorro = dmc.DataAndMetadataCats(directorio_dp,recurso)

In [ ]:
directorio_resultados = '../tmp/'
archivo = 'distribucion_posterior_repeticion_2_iteraciones_1000000.csv'
datos = pd.read_csv(directorio_resultados + archivo)

In [ ]:
# n_datos = 34
n_datos = len(DatosSocorro.getData())
nombre_esfuerzo = DatosSocorro.glosario["esfuerzo"]
nombre_capturas = DatosSocorro.glosario["capturas"]

Creo que aquí se tiene que hacer un cambio de unidades en el esfuerzo

In [ ]:
esfuerzo = np.array(DatosSocorro.getValue(nombre_esfuerzo)/(30 * 7 * 5))[0:n_datos]  #Días hombre
capturas = np.array(DatosSocorro.getValue(nombre_capturas))[0:n_datos]

## Tamaño inicial de la población
Aquí se grafica la distribucón posterior para el tamaño inicial de la población $N_o$.

In [ ]:
No = datos.No

In [ ]:
bins_necesarios = max(No) - min(No)
histograma_completo, lados, otracosa = plt.hist(No, bins=bins_necesarios, normed=1)
plt.xlim(min(No),min(No)+200)
plt.show()

## Gatos remanentes
Histograma de los gatos remanentes. Es el histograma anterior pero el eje $x$ recorrido el número total de gatos capturados. 

In [ ]:
bins_trasladados = lados - sum(capturas)
bins_centrados = (bins_trasladados[:-1] + bins_trasladados[1:])/2

In [ ]:
remanentes = No - sum(capturas)
ic = np.percentile(np.asarray(remanentes), [2.5, 97.5])
ic

In [ ]:
intervaloDensidadPosteriorSuperior = np.asarray(pe.calc_min_interval(np.sort(remanentes.values),alpha=0.95))
intervaloDensidadPosteriorSuperior

In [ ]:
vmp = intervaloDensidadPosteriorSuperior.mean().round()
vmp

## Función de densidad de probabilidad

In [ ]:
esHPD = (bins_centrados<=intervaloDensidadPosteriorSuperior.max()) & (intervaloDensidadPosteriorSuperior.min()<=bins_centrados)
indVMP = np.argmin(abs(bins_centrados - vmp))
esInferior = (bins_centrados<ic.min())
esSuperior = (bins_centrados>ic.max())

In [ ]:
bi.curdoc().clear()
bp.reset_output()
bp.output_file('../resultados/gatos_remanentes.html', title="Gatos remanentes")

In [ ]:
integral = histograma_completo.cumsum()
fuente = bp.ColumnDataSource({'x': bins_centrados, 'p': histograma_completo, 'c': integral})
encima_pdf = bm.HoverTool(tooltips=[
    ("Cantidad de gatos remanentes", "@x"),
    ("Probabilidad", "@p")],
    mode='vline')
encima_cdf = bm.HoverTool(tooltips=[
    ("Cantidad de gatos remanentes", "@x"),
    ("Probabilidad", "@c")],
    mode='vline')
herramientas = "pan,box_zoom,wheel_zoom,resize,reset,save"

In [ ]:
pdf = bp.figure(title="PDF de gatos remanentes", x_axis_label='Cantidad de gatos remanentes', y_axis_label='Probabilidad', \
            y_range=(0,0.035), tools=[encima_pdf, herramientas])

pdf.vbar(x = bins_centrados, top=histograma_completo, bottom=0, width = 1)
pdf.line([vmp, vmp], [0, histograma_completo.max()], line_width=5, line_color="DarkOrange")
pdf.vbar(x = bins_centrados[esInferior], top=histograma_completo[esInferior], bottom=0, width = 1, fill_color="SeaGreen", line_color="SeaGreen")
pdf.vbar(x = bins_centrados[esSuperior], top=histograma_completo[esSuperior], bottom=0, width = 1, fill_color="SeaGreen", line_color="SeaGreen")
pdf.line('x','p', source=fuente, line_alpha=0)
pdf.grid.visible = False
indUltimoInferior = np.argmax(~esInferior);
indPrimerSuperior = np.argmax(esSuperior);
pdf.text(x = [ic[0]], y=[histograma_completo[indUltimoInferior]], text=[ic[0]], text_align='right')
pdf.text(x = [ic[1]], y=[histograma_completo[indPrimerSuperior]], text=[ic[1]], text_align='left')
pdf.text(x = [vmp], y=[histograma_completo[indVMP]+0.001], text=[vmp], text_align='center', text_baseline = 'bottom')

## Función de probabilidad acumulativa

In [ ]:
cdf = bp.figure(title="CDF de gatos remanentes", x_axis_label="Número de gatos remanentes",\
            y_axis_label='Probabilidad',y_range=(0, 1), x_range=(0,150), tools=[encima_cdf, herramientas])
cdf.grid.visible = False
cdf.line('x','c', line_width=3, source=fuente)
cdf.line([bins_centrados[indPrimerSuperior], bins_centrados[indPrimerSuperior]],[0, integral[indPrimerSuperior]], line_width=3, line_color="SeaGreen")
cdf.line([bins_centrados[indUltimoInferior], bins_centrados[indUltimoInferior]],[0, integral[indUltimoInferior]], line_width=3, line_color="SeaGreen")
fuenteVMP = bp.ColumnDataSource({'x': [vmp], 'p': [histograma_completo.max()], 'c': [integral[indVMP]]})
cdf.circle('x', 'c', size=10, color="DarkOrange", source=fuenteVMP)
cdf.line([vmp, vmp], [0, integral[indVMP]], line_width=3, color="DarkOrange")


In [ ]:
p = bp.gridplot([[pdf, cdf]])
bp.show(p)